In [1]:
import pathlib
import time
import zipfile
from multiprocessing import Pool, Lock, Manager, cpu_count, current_process
from concurrent.futures import ThreadPoolExecutor, as_completed
import platform

# Задача 1. Удвоение чисел и получение первого результата

In [2]:
# Получение данных из файла
data_path = pathlib.Path.cwd().joinpath('data/test_list_numbers.txt')
# Считаем содержимое файла в одну строку избавившись от переводов строк и пробелов
data_str = ""
with open(data_path, 'r') as f:
    for line in f.readlines():
        data_str += line.replace('\n','').replace(' ', '')
# убрать внешние скобки - останется только "содержимое" внешнего списка
data_str = data_str[1:-1] 
# разделить на "вложеные" списки
data_list = data_str.replace("[","").split("],")

def str_to_int_list(str_with_numbers):
    "Строку с числами преобразовать список чисел"
    l1 = str_with_numbers.split(",")
    s2i = lambda s: int(s) if s.isdigit() else None
    return list(map(s2i, l1))
# преобразовать в списки с числами
data_int = list(map(str_to_int_list,data_list))

In [3]:
class ProcessNumberList:
    """Организация параллельной обработки списков чисел"""
    # флаг, того надо или нет прервать процесс обработки списков, выполняемых обработчиками конкретного экземпляра класса
    __need_stop_process_list = False

    def __init__(self):
        self.reset_need_stop_process_list()
        
    @property
    def need_stop_process_list(self):
        """Флаг необходимости прерывания соседних потоков, обрабатываемых в этом же объекте"""
        return self.__need_stop_process_list

    def reset_need_stop_process_list(self):
        """Сбросить флаг необходимости прерывания соседних потоков"""
        self.__need_stop_process_list = False

    def __process_number(self, number):
        """Обработка одно числа из списка"""
        result = number * 2
        time.sleep(0.1) # задержка выше, чем в задании, чтобы нагляднее была разница в длительности двух подходов
        return result

    def __process_list(self, numbers, break_after_first = False):
        """Обработка списка чисел"""
        result = 0
        for number in numbers:
            if break_after_first and self.__need_stop_process_list:
                # соседний поток обработал свой список - значит завершить и обработку текущего списка
                return None
            if number is not None:
                result += self.__process_number(number)
        if break_after_first:
            # текущий список завершили обрабатывать - сообщить об этом соседним потокам
            self.__need_stop_process_list = True
        return result

    def process_list_break_after_first(self, numbers):
        """Обработать список чисел, остановившись как только какой-нибудь поток обработает свой список"""
        return self.__process_list(numbers, break_after_first = True)

    def process_list_wait_all(self, numbers):
        """Обработать список чисел не обращая внимания на соседние потоки"""
        return self.__process_list(numbers, break_after_first = False)    


In [4]:
%%time
# Вариант без принудительного останова параллельных потоков
first_list_sum = None
# Запуск без принудительного останова параллельных потоков
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    p = ProcessNumberList()
    futures = [executor.submit(p.process_list_wait_all, l) for l in data_int]

    # Получение результатов задач
    for future in as_completed(futures):
        first_list_sum = future.result()
        break # больше результатов можно не ждать - прерываем цикл 
print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")

Сумма чисел в первом обработанном списке: 11090
CPU times: user 6.34 ms, sys: 4.21 ms, total: 10.5 ms
Wall time: 2.81 s


In [5]:
%%time
# Вариант с принудительным остановом обработки в параллельных потоках
from concurrent.futures import ThreadPoolExecutor, as_completed
first_list_sum = None
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    p = ProcessNumberList()
    futures = [executor.submit(p.process_list_break_after_first, l) for l in data_int]

    # Получение результатов задач
    for future in as_completed(futures):
        first_list_sum = future.result()
        break # больше результатов можно не ждать - прерываем цикл 
print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")

Сумма чисел в первом обработанном списке: 11090
CPU times: user 2.45 ms, sys: 7.5 ms, total: 9.95 ms
Wall time: 1.3 s


# Задача 2. Подсчет суммы на основе значений из файлов, ссылки на которых находятся в других файлах

In [6]:
def prepared_filename(filename):
    """Преобразование стиль пути к файлу Linux <-> Windows"""
    if platform.system()=='Linux':
        filename = filename.replace('\\','/')    
    else:
        filename = filename.replace('/', '\\')    
    return filename

In [7]:
zip_with_filenames_path = pathlib.Path.cwd().joinpath('data/path_8_8.zip')
data_zip_path = pathlib.Path.cwd().joinpath('data/recursive_challenge_8_8.zip')

## Вариант-1. Zip-файлы открываются каждый раз в параллельной задаче

In [9]:
def process_file(filename, zip_with_filenames_path, data_zip_path):
    """Обработать один файл"""
    
    # считаем ссылку на файл с данными из нужного файла в архиве с файлами-ссылками
    with zipfile.ZipFile(zip_with_filenames_path) as zip_with_filenames:
        data_file_path = prepared_filename(zip_with_filenames.read(filename).decode())
    # найти файл в архиве с данными и считать из него число
    with zipfile.ZipFile(data_zip_path) as data_zip:
        num = int(data_zip.read(data_file_path).decode())
    return num


In [ ]:
# Получение списка файлов
files1 = []
with zipfile.ZipFile(zip_with_filenames_path) as paths_file:
    files1 = [(p.filename, zip_with_filenames_path, data_zip_path) for p in paths_file.infolist() if not p.is_dir()]

In [ ]:
%%time
# Однопоточная обработка файлов (для сравнения)
total_sum = 0
for f in files1:
    total_sum += process_file(f[0], f[1], f[2])
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 3min 42s, sys: 12.5 s, total: 3min 55s
Wall time: 3min 55s


In [ ]:
%%time 
# Многопроцессная обработка файлов через multiprocessing.Pool
total_sum = 0
with Pool(processes=10) as pool:
    for r in pool.starmap(process_file, files1):
        total_sum += r
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 14.8 ms, sys: 32 ms, total: 46.8 ms
Wall time: 49.1 s


In [ ]:
%%time
# Многопоточная обработка файлов через ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor, as_completed
total_sum = 0
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    futures = [executor.submit(process_file, f[0], f[1], f[2]) for f in files1]

    # Получение результатов задач
    for future in as_completed(futures):
        total_sum +=  future.result()
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 4min 26s, sys: 5.75 s, total: 4min 31s
Wall time: 4min 21s


## Вариант-2. Файлы предварительно распакованы

In [14]:
files_with_filenames_path = pathlib.Path.cwd().joinpath('extracted_data/path')
data_files_path = pathlib.Path.cwd().joinpath('extracted_data')

In [79]:
# Получение списка файлов-ссылок
files4 = []
for _, _, filenames in files_with_filenames_path.walk():
    pass
files4 = [(filename, files_with_filenames_path, data_files_path) for filename in filenames]
files2 = []
files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)
# files2.extend(files4)


In [45]:
def process_file2(filename, files_with_filenames_path, data_files_path):
    """Обработать один файл"""
    # считаем ссылку на файл с данными из нужного файла в архиве с файлами-ссылками
    with open(pathlib.Path(files_with_filenames_path).joinpath(filename), 'r') as file_with_link:
        relative_link = file_with_link.readline()
    with open(pathlib.Path(data_files_path).joinpath(prepared_filename(relative_link)), 'rb') as data_file:
        num = int(data_file.readline())
    return num

In [83]:
%%time
# Однопоточная обработка файлов (для сравнения)
total_sum = 0
for f in files2:
    total_sum += process_file2(f[0], f[1], f[2])
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 16.1 ms, sys: 14 ms, total: 30.1 ms
Wall time: 29.8 ms


In [87]:
%%time 
# Многопроцессная обработка файлов через multiprocessing.Pool
total_sum = 0
with Pool(processes=10) as pool:
    for r in pool.starmap(process_file2, files2):
        total_sum += r
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 6.56 ms, sys: 46.6 ms, total: 53.1 ms
Wall time: 64 ms


In [95]:
%%time
# Многопоточная обработка файлов через ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor, as_completed
total_sum = 0
with ThreadPoolExecutor(max_workers=10) as executor:
#with ThreadPoolExecutor() as executor:
    # Создание и запуск задач
    futures = [executor.submit(process_file2, f[0], f[1], f[2]) for f in files2]

    # Получение результатов задач
    for future in as_completed(futures):
        total_sum +=  future.result()
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 183 ms, sys: 112 ms, total: 295 ms
Wall time: 196 ms


## Вариант-3. Своя копия открытого zip-файла для каждого процесса

In [118]:
files4 = []
with zipfile.ZipFile(zip_with_filenames_path) as paths_file:
    files4 = [(p.filename, ) for p in paths_file.infolist() if not p.is_dir()]
files3 = []
files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)
# files3.extend(files4)

In [106]:
def initialize(zip_with_filenames_path, data_zip_path):
    global zip_with_filenames_dict
    global data_zip_dict
    zip_with_filenames_dict[current_process().name] = zipfile.ZipFile(zip_with_filenames_path)
    data_zip_dict[current_process().name] = zipfile.ZipFile(data_zip_path)

In [107]:
def process_file3(filename):
    """Обработать один файл"""

    global zip_with_filenames_dict
    global data_zip_dict
    
    # считаем ссылку на файл с данными из нужного файла в архиве с файлами-ссылками
    with zip_with_filenames_dict[current_process().name].open(filename) as file_with_link:
        data_file_path = prepared_filename(file_with_link.read().decode())

    # найти файл в архиве с данными и считать из него число
    with data_zip_dict[current_process().name].open(data_file_path) as data_file:
        num = int(data_file.readline().decode())

    return num


In [122]:
%%time
# Однопоточная обработка файлов (для сравнения)
total_sum = 0
zip_with_filenames_dict = {}
data_zip_dict = {}
initialize(zip_with_filenames_path, data_zip_path)
for f in files3:
    total_sum += process_file3(f[0])
zip_with_filenames_dict = {}
data_zip_dict = {}
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 232 ms, sys: 3.99 ms, total: 236 ms
Wall time: 235 ms


In [ ]:
%%time
# Многопроцессная обработка файлов через multiprocessing.Pool
total_sum = 0
zip_with_filenames_dict = {}
data_zip_dict = {}
with Pool(processes=10, initializer=initialize, initargs=(zip_with_filenames_path, data_zip_path)) as pool:
    for r in pool.starmap(process_file3, files3):
        total_sum += r
zip_with_filenames_dict = {}
data_zip_dict = {}
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 4.95 ms, sys: 9.17 ms, total: 14.1 ms
Wall time: 295 ms


In [ ]:
%%time
# Многопоточная обработка файлов через ThreadPoolExecutor
total_sum = 0
zip_with_filenames_dict = {}
data_zip_dict = {}
initialize(zip_with_filenames_path, data_zip_path)
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    futures = [executor.submit(process_file3, f[0]) for f in files3]
    # Получение результатов задач
    for future in as_completed(futures):
        total_sum +=  future.result()
zip_with_filenames_dict = {}
data_zip_dict = {}
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 377 ms, sys: 41 ms, total: 418 ms
Wall time: 381 ms
